In [2]:
from hhsolver import *
import aspose.words as aw
from PIL import Image

In [3]:
%load_ext autoreload


In [4]:
%autoreload 2

In [15]:
# HHsampling = função para gerar as alocações com o método do HH (haphazard)

# x is transformed data
# lamb is pure lambda in [0,1]
# sample_size is number of elements to sample
# noise is list of size n, alloc with transformed noise matrices
# returns a dictionary with entries:
#   allocs: matrix n x n alloc where each column j is an allocation associated to noise j
#   distq95: 95th percentile of Mahalanobis distances for each allocation using only original data x
#   fkappa: fleiss kappa of allocations
def hhsampling(x, lamb, nalloc, sample_size, noise, populations, hh_solver_strategy):

    ma = noise[0].shape[1] # number of columns in noise matrix
    m = x.shape[1]

    res = {"allocs": None, "distq95": 0, "fkappa": 0, "dist": None}

    # CHECK IF LAMBDA IS CORRECT
    lamb = lamb/(lamb*(1-ma/m) + ma/m)

    dist = []

    allocs = np.zeros((nalloc, x.shape[0]), dtype=int)

    for j in range(nalloc):

        xz = np.concatenate(((1-lamb)*x, lamb*noise[j]), axis=1)

        solver = hh_solver_strategy(xz, sample_size) # 
        solver.init_pop = populations[j] # generate_pop(50, solver.sample_size, solver.data.shape[0]) # !!

        w_index = solver.get_best_alloc()
        # print(w_index)
        # d = obj_func(w_index,x)
        # allocs[j][w_index] = 1
        # dist.append(d)

    # res["allocs"] = allocs.T
    # res["dist"] = dist
    # distq95 = np.percentile(np.array(dist), 100)
    
    # res["distq95"] = distq95
    # fkappaval = fleiss_kappa_allocs(allocs.T)
    # res["fkappa"] = fkappaval
    
    return res


In [10]:
file_path = '../data/Candidatos20230803_n.txt'
data = collect_data(file_path=file_path).to_numpy()

lambda_n = 0#, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

nalloc = 2

sample_size = [60]#[60, 120, 180, 240] 

fileNames = []

# hhsamplingGA require transformed data
x = data @ inversecholcov(data)
solver = HHSolverPSO

In [24]:
# Teste com lambda 0
# fig, axs = plt.subplots(len(sample_size), 1, figsize=(8, 10), sharex=True)
# min = []
for index, sample in enumerate(sample_size):
        # pre-generate noises used in each batch
        ma = 2
        noise = []
        populations = []
        for i in range(nalloc):
            z = generate_noise(x.shape[0], ma)
            # hhsamplingGA require transformed noise
            zt = z @ inversecholcov(z)
            noise.append(zt)
            populations.append(generate_pop(50, sample, x.shape[0]))

        stats_aux = {"distq95": [], "fkappa": []}

        #for i in range(len(lambdas)):
        res = hhsampling(x, lambda_n, nalloc, sample, noise, populations, solver)
        for k in stats_aux:
            stats_aux[k].append(res[k])
        
        # dist = list(np.array(res["dist"]))
        # plt.subplot(len(sample_size), 1, index+1)
        
        # plt.hist(x = dist,bins=30)
        # min.append(min(dist))
        # plt.title(f'Sample Size: {sample}') 
       
# plt.tight_layout()
# plt.show()


False
False
